<a href="https://colab.research.google.com/github/bijuthottathil/dataengineer/blob/main/Realtime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
!tar xf spark-3.0.0-bin-hadoop3.2.tgz
!pip -q install findspark


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages io.delta:delta-core_2.12:0.7.0 --conf spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension --conf spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog pyspark-shell'
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark=SparkSession.builder.appName('Myapp').getOrCreate()

In [ ]:
data=spark.read.csv("/content/emp-pyspark.csv",inferSchema=True,header=True)

In [ ]:
data.show()

+-----+------+---------+----+----------+----+----+------+------------+
|EMPNO| ENAME|      JOB| MGR|  HIREDATE| SAL|COMM|DEPTNO|UPDATED_DATE|
+-----+------+---------+----+----------+----+----+------+------------+
| 7369| SMITH|    CLERK|7902|17-12-1980| 800|null|    20|  2022-01-01|
| 7499| ALLEN| SALESMAN|7698|20-02-1981|1600| 300|    30|  2022-01-01|
| 7521|  WARD| SALESMAN|7698|22-02-1981|1250| 500|    30|  2022-01-01|
| 7566| JONES|  MANAGER|7839|04-02-1981|2975|null|    20|  2022-01-05|
| 7654|MARTIN| SALESMAN|7698|21-09-1981|1250|1400|    30|  2022-01-03|
| 7698|   SGR|  MANAGER|7839|05-01-1981|2850|null|    30|  2022-01-04|
| 7782|  RAVI|  MANAGER|7839|06-09-1981|2450|null|    10|  2022-01-02|
| 7788| SCOTT|  ANALYST|7566|19-04-1987|3000|null|    20|  2022-01-02|
| 7839|  KING|PRESIDENT|null|01-11-1981|5000|null|    10|  2022-01-02|
| 7844|TURNER| SALESMAN|7698|09-08-1981|1500|   0|    30|  2022-01-02|
| 7876| ADAMS|    CLERK|7788|23-05-1987|1100|null|    20|  2022-01-03|
| 7900

In [ ]:
# How to create partition by month and year in pyspark
from pyspark.sql.functions import *
emp_csv=data.withColumn("HIREDATE",to_date("HIREDATE",'dd-MM-yyyy')).fillna({"HireDate":"9999-12-31"})
emp_csv=emp_csv.withColumn("Month",month("HIREDATE")).withColumn("Year",year("HIREDATE"))

In [ ]:
from delta.tables import *
emp_csv.write.format("delta").partitionBy("Year","Month").mode("overwrite").saveAsTable("emp_csvnew1")

In [ ]:
deltainstance=DeltaTable.forPath(spark,"/content/spark-warehouse/emp_csvnew1/Year=1980")
deltainstance.toDF().show()

+-----+-----+-----+----+----------+---+----+------+------------+-----+----+
|EMPNO|ENAME|  JOB| MGR|  HIREDATE|SAL|COMM|DEPTNO|UPDATED_DATE|Month|Year|
+-----+-----+-----+----+----------+---+----+------+------------+-----+----+
| 7369|SMITH|CLERK|7902|1980-12-17|800|null|    20|  2022-01-01|   12|1980|
| 7369|SMITH|CLERK|7902|1980-12-17|800|null|    20|  2022-01-04|   12|1980|
+-----+-----+-----+----+----------+---+----+------+------------+-----+----+

